# 04 · Fine-tune Slice *t* (Unsloth QLoRA)

Load 4-bit Llama weights, configure adapters, and respect the 25M token budget per slice.

In [1]:
!pip install unsloth

In [2]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)


PROJECT_ROOT: /content/drive/MyDrive/secure-llm-mia
Active run mode: subset - 30k-example subset powering the 4-slice, 3M-token continual fine-tuning regime.
BHC CSV path: /content/drive/MyDrive/mimic-iv-bhc/mimic-iv-bhc.csv


In [3]:
import math
import os
from pathlib import Path
from typing import Dict

import torch
import pandas as pd
from datasets import Dataset, concatenate_datasets

from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer, SFTConfig

from src.modeling.lora import LoRAHyperParams, compute_gradient_accumulation
from src.modeling.train import TokenBudgetTracker

# ---------------------------------------------------------------
# 1. Load sliced TEXT dataset produced in notebook 02
# ---------------------------------------------------------------

SLICED_PATH = ARTIFACTS_DIR / f"sliced_dataset_{RUN_MODE.name}.parquet"
if not SLICED_PATH.exists():
    raise FileNotFoundError(
        f"Missing sliced text dataset {SLICED_PATH}. Run notebook 02 first."
    )

df_sliced = pd.read_parquet(SLICED_PATH)
print("Loaded sliced rows:", len(df_sliced))

# Build slice -> dataset mapping (train only)
slice_datasets: Dict[int, Dataset] = {}
for sid in sorted(df_sliced["slice_id"].unique()):
    slice_df = df_sliced[(df_sliced["slice_id"] == sid) & (df_sliced["split_tag"] == "train")].copy()
    ds = Dataset.from_pandas(slice_df, preserve_index=False)
    if "text" not in ds.column_names:
        raise ValueError("Dataset must contain a `text` column.")
    slice_datasets[int(sid)] = ds
    print(f"Slice {sid}: {len(ds)} training rows loaded (text).")

SLICES = sorted(slice_datasets.keys())
TRACKS = ["noreplay", "replay10"]
REPLAY_FRACTION = 0.10

# ---------------------------------------------------------------
# 2. Compute token-budget-driven step counts
# ---------------------------------------------------------------

MODEL_NAME = os.getenv("UNSLOTH_MODEL_NAME", "unsloth/Llama-3.2-3B-bnb-4bit")
MAX_SEQ_LENGTH = 4096
TOKENS_PER_SLICE = 3_000_000
TOKENS_PER_STEP = 128_000
MICRO_BATCH = 1
AVG_TOKENS_PER_SAMPLE = 3000

accum_steps = compute_gradient_accumulation(
    TOKENS_PER_STEP, MICRO_BATCH, AVG_TOKENS_PER_SAMPLE
)
print("Gradient accumulation:", accum_steps)

max_steps = math.ceil(TOKENS_PER_SLICE / TOKENS_PER_STEP)
print("Max steps per slice:", max_steps)

is_ampere_plus = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8
use_bf16 = bool(is_ampere_plus and torch.cuda.is_bf16_supported())
use_fp16 = torch.cuda.is_available() and not use_bf16


# ---------------------------------------------------------------
# 3. Unsloth model initializer
# ---------------------------------------------------------------

def init_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_NAME,
        max_seq_length=MAX_SEQ_LENGTH,
        load_in_4bit=True,
    )

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    tokenizer.model_max_length = MAX_SEQ_LENGTH

    lora_cfg = LoRAHyperParams(
        r=32,
        alpha=32,
        dropout=0.0,
        target_modules=(
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj"
        ),
    )
    model = FastLanguageModel.get_peft_model(
        model,
        r=lora_cfg.r,
        target_modules=list(lora_cfg.target_modules),
        lora_alpha=lora_cfg.alpha,
        lora_dropout=lora_cfg.dropout,
        bias="none",
        use_gradient_checkpointing="unsloth",
    )

    FastLanguageModel.for_training(model)
    return model, tokenizer


# ---------------------------------------------------------------
# 4. Training loop: continual finetuning across slices per track
# ---------------------------------------------------------------

for track in TRACKS:
    print(f"=== Track: {track} ===")

    # Initialise base model ONCE per track
    model, tokenizer = init_model()

    for slice_id in SLICES:
        # Base slice data
        base_ds = slice_datasets[slice_id]
        train_ds = base_ds

        # Optional replay: add a fraction of previous slices' data
        if track == "replay10":
            prior_slice_ids = [sid for sid in SLICES if sid < slice_id]
            if prior_slice_ids:
                combined = concatenate_datasets([slice_datasets[p] for p in prior_slice_ids])
                replay_n = min(int(len(base_ds) * REPLAY_FRACTION), len(combined))
                if replay_n > 0:
                    replay_subset = combined.shuffle(seed=17).select(range(replay_n))
                    train_ds = concatenate_datasets([base_ds, replay_subset])
                    print(
                        f"Slice {slice_id}: added {replay_n} replay samples from previous slices."
                    )

        # Directory for this slice's checkpoint
        output_dir = CHECKPOINT_ROOT / f"slice_{slice_id}" / track / RUN_MODE.name
        output_dir.mkdir(parents=True, exist_ok=True)

        # SFT config (one run per slice, continuing from current model weights)
        sft_config = SFTConfig(
            output_dir=str(output_dir),
            per_device_train_batch_size=MICRO_BATCH,
            gradient_accumulation_steps=accum_steps,
            max_seq_length=MAX_SEQ_LENGTH,
            warmup_steps=max(1, int(0.1 * max_steps)),
            max_steps=max_steps,
            learning_rate=5e-5,
            logging_steps=1,
            save_steps=50,
            bf16=use_bf16,
            fp16=use_fp16,
            optim="adamw_8bit",
            lr_scheduler_type="linear",
            weight_decay=0.01,
        )

        trainer = SFTTrainer(
            model=model,              # reuse the SAME model object
            tokenizer=tokenizer,
            train_dataset=train_ds,
            args=sft_config,
            dataset_text_field="text",
            packing=False,
            dataset_num_proc=2,
        )

        print(f"--- Training slice {slice_id} ({track}) ---")
        trainer.train()

        # Keep updated weights in `model` for the next slice
        model = trainer.model

        # Approx token accounting (for logging)
        approx_tokens = len(train_ds) * AVG_TOKENS_PER_SAMPLE
        print(f"Slice {slice_id}: approx tokens consumed {approx_tokens:,}")

        # Save LoRA checkpoint for this slice
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        print(f"Saved adapters + tokenizer to {output_dir}")

        del trainer
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # (optional) after all slices for this track:
    del model, tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loaded sliced rows: 4225
Slice 1: 1020 training rows loaded (text).
Slice 2: 1062 training rows loaded (text).
Slice 3: 1094 training rows loaded (text).
Slice 4: 1049 training rows loaded (text).
Gradient accumulation: 43
Max steps per slice: 24
=== Track: noreplay ===
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1020 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


--- Training slice 1 (noreplay) ---


...

The model is already on multiple devices. Skipping the move to device specified in `args`.


--- Training slice 4 (replay10) ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,153 | Num Epochs = 1 | Total steps = 24
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 43
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 43 x 1) = 43
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)


Step,Training Loss
1,1.706600
2,1.761000
3,1.705800
4,1.713900
5,1.687000
6,1.668300
7,1.648200
8,1.712900
9,1.698900
10,1.685700


Slice 4: approx tokens consumed 3,459,000
Saved adapters + tokenizer to /content/drive/MyDrive/secure-llm-mia/checkpoints/slice_4/replay10/subset
